## Lab 5: SQL

In this lab, we'll explore using SQL with pandas. 

In [1]:
import sqlalchemy
import pandas as pd

engine = sqlalchemy.create_engine("sqlite:///lab05.sqlite")
connection = engine.connect()

## Rapidgram

The date: February, 2020. All of the students at UChicago are obsessed with the hot new social networking app, Rapidgram, where users can share text and image posts. You've been hired as Rapidgram's very first Data Scientist, in charge of analyzing their petabyte-scale user data, in order to sell it to credit card companies (I mean, they had to monetize somehow). But before you get into that, you need to learn more about their database schema.

The next few cells will generate a snapshot of their data. It will be saved locally as the file `lab05.sqlite`. 

First, we create empty tables to store the data.

In [2]:
sql_expr = """
DROP TABLE IF EXISTS users;
"""
result = engine.execute(sql_expr)

sql_expr = """
DROP TABLE IF EXISTS follows;
"""
result = engine.execute(sql_expr)

sql_expr = """
CREATE TABLE users (
    USERID   INT            NOT NULL,                 
    NAME     VARCHAR (256)  NOT NULL,                 
    YEAR     FLOAT          NOT NULL,
    PRIMARY KEY (USERID)
);
"""
result = engine.execute(sql_expr)

sql_expr = """
CREATE TABLE follows (
    USERID   INT            NOT NULL, 
    FOLLOWID INT            NOT NULL,
    PRIMARY KEY (USERID, FOLLOWID)
);
"""
result = engine.execute(sql_expr)

Next, we insert some users into the `users` table. If you haven't seen the `str.format()` method before, take a look at the [docs](https://docs.python.org/3.4/library/stdtypes.html#str.format):

In [3]:
count = 0
users = ["Ian", "Daniel", "Sarah", "Kelly", "Sam", "Alison", "Henry", "Joey", "Mark", "Joyce", "Natalie", "John"]
years = [1, 3, 4, 3, 4, 2, 5, 2, 1, 3, 4, 2]

for username, year in zip(users, years):
    count += 1
    sql_expr = """
    INSERT INTO users 
    VALUES ({}, '{}', {});
    """.format(count, username, year)
    engine.execute(sql_expr)

Let's see what the table looks like:

In [ ]:
sql_expr ="""
SELECT *
FROM users
"""
pd.read_sql(sql_expr, engine)

Now, we'll generate some data to keep track of who follows who.

In [4]:
follows =  [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 
            0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 
            0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 
            1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 
            0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
            0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
            1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
            1, 1, 0, 1]

for i in range(12):
    for j in range(12):
        if i != j and follows[i + j*12]:
            sql_expr = """
            INSERT INTO follows 
            VALUES ({}, {});
            """.format(i+1, j+1)
            engine.execute(sql_expr)

Let's look at this data now:

In [5]:
sql_expr ="""
SELECT *
FROM follows
"""
pd.read_sql(sql_expr, engine).head()

,USERID,FOLLOWID
0,1,2
1,1,4
2,1,6
3,1,8
4,1,11


Great! We have our databases all setup. If it isn't clear, the USERID from the `users` table allows you to figure out who follows who using the `follows` table. For example, the first row of the `follows` table (USERID=1, FOLLOWID=2) tells us that Daniel follows Ian.

#### Question 1: Joey's Followers

Write a SQL query to determine how many people follow Joey.

In [ ]:
q1 = """
...
"""

pd.read_sql(q1, engine)

<details><summary><button>Click here to reveal the answer!</button></summary>
q1 = """ 
SELECT COUNT(*) FROM follows, users 
    WHERE users.name="Joey" 
    AND (users.userid=follows.followid)
"""
</details>

#### Question 2: I Ain't no Followback Girl

How many people does Joey follow?

In [ ]:
q2 = """
...
"""
pd.read_sql(q2, engine)

<details><summary><button>Click here to reveal the answer!</button></summary>
q2 = """ 
SELECT COUNT(*) FROM follows, users
    WHERE users.name="Joey"
    AND (users.userid=follows.userid)
"""
</details>

#### Question 3: Know your Audience
What are the names of Joey's followers?

In [ ]:
q3 = """
...
"""
pd.read_sql(q3, engine)

<details><summary><button>Click here to reveal the answer!</button></summary>
q3 = """
SELECT u1.name
    FROM follows, users as u1, users as u2
    WHERE follows.userid=u1.userid
    AND follows.followid=u2.userid
    AND u2.name="Joey"
"""
</details>

#### Question 4: Popularity Contest

How many followers does each user have? You'll need to use `GROUP BY` to solve this. List only the top 5 users by number of followers.

In [ ]:
q4 = """
...
"""
pd.read_sql(q4, engine)

<details><summary><button>Click here to reveal the answer!</button></summary>
q4 = """
SELECT name, COUNT(*) as friends
    FROM follows, users
    WHERE follows.followid=users.userid
    GROUP BY name
    ORDER BY friends DESC
    LIMIT 5
"""
</details>

#### Question 5: Randomness

Rapidgram wants to get a random sample of their userbase. Specifically, they want to look at *exactly* one-third of the follow-relations in their data. A Rapidgram engineer suggests the following SQL query:

In [ ]:
q5a = """
SELECT u1.name as follower, u2.name as followee
    FROM follows, users as u1, users as u2
    WHERE follows.userid=u1.userid
    AND follows.followid=u2.userid
    AND RANDOM() < 0.33
"""

Do you think this query will work as intended? Why or why not? Try designing a better query below:

In [ ]:
q5b = """
...
"""
pd.read_sql(q5b, engine)

<details><summary><button>Click here to reveal the answer!</button></summary>
q5b = """
SELECT u1.name as follower, u2.name as followee
    FROM follows, users as u1, users as u2
    WHERE follows.userid=u1.userid
    AND follows.followid=u2.userid
    ORDER BY RANDOM() LIMIT 72*1/3
"""
</details>

#### Question 6: Older and Wiser (challenge)
List every person who has been at Berkeley longer - that is, their `year` is greater - than their average follower.

In [ ]:
q6 = """
SELECT name FROM (
    SELECT ...
)
WHERE year > avg_follower_years
"""
pd.read_sql(q6, engine)

<details><summary><button>Click here to reveal the answer!</button></summary>
q6 = """
SELECT name FROM 
    (SELECT u1.name, u1.year, AVG(u2.year) as avg_follower_years
        FROM follows, users as u1, users as u2
        WHERE follows.userid=u1.userid
        AND follows.followid=u2.userid
        GROUP BY u1.name)
    WHERE year > avg_follower_years
"""
</details>

## Lab 5 Done!

The materials for this lab have been sourced from [Data 100 at UC Berkeley](http://www.ds100.org/)